# Sequential Chains

In LangChain, a **RunnableSequence** allows you to link multiple steps together in a specific order, creating a "chain". The output of one step becomes the input for the next step.

This is similar to a factory assembly line where raw material (input) passes through stations (prompt, model, parser) to become a finished product.

In [ ]:
# Sequential runnable

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Give some intresting facts about {topic}',
    input_variables=['topic']
)

chain = RunnableSequence(prompt1, model, parser)
result = chain.invoke({'topic': 'AI'})
print(result)

# Parallel Execution

**RunnableParallel** lets you run multiple independent tasks simultaneously based on the same input.

For example, from a single topic, you can generate a summary, key notes, and facts all at once. The result is a dictionary containing the outputs from all parallel branches.

In [ ]:
# Parallel Runnable

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

# Artical research
prompt1 = PromptTemplate(
    template='Generate 250 words summary on {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Generate Bullet-point-notes on {topic}. cover only important points and make it concise.',
    input_variables=['topic']
)
prompt3 = PromptTemplate(
    template='Generate key-facts on {topic}',
    input_variables=['topic']
)
prompt4 = PromptTemplate(
    template='tell some possible applications on {topic}',
    input_variables=['topic']
)
prompt5 = PromptTemplate(
    template='Generate a Detailed Report from the content: \nSummary: {summary} \nPoint-Notes: {notes} \nKey-Facts: {facts} \nApplication: {application}',
    input_variables=['summary', 'notes', 'facts', 'application']
)

parallel_chain = RunnableParallel({
    'summary': RunnableSequence(prompt1, model, parser),
    'notes': RunnableSequence(prompt2, model, parser),
    'facts': RunnableSequence(prompt3, model, parser),
    'application': RunnableSequence(prompt4, model, parser)
})

chain = RunnableSequence(parallel_chain, prompt5, model, parser)
print(chain.invoke({'topic':'Agentic AI'}))



# Custom Data Flow and Logic

As chains grow complex, you need precise control over data.

*   **RunnablePassthrough**: Allows you to pass input data to later steps or add new keys to the existing data flow without overwriting everything.
*   **RunnableLambda**: Wraps standard Python functions so they can be used as steps in a chain (e.g., for counting words or formatting text).

In [ ]:
# runnable Passthrough and Lambda
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Explain {topic} in siple words.',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Give some key-points about {topic}',
    input_variables=['topic']
)
prompt3 = PromptTemplate(
    template='Generate some relivent question-answers about {topic}',
    input_variables=['topic']
)
chain = RunnableSequence(prompt1, model, parser)
parallel_chain = RunnableParallel({
    "points": RunnableSequence(prompt2, model, parser),
    'quiz': RunnableSequence(prompt3, model, parser ),
    'words': RunnableLambda(lambda x : len(x.split()))
})
pipeline = RunnableSequence(chain, parallel_chain)
result = pipeline.invoke({'topic': 'AI'})
print(result.keys())

print()
print(chain.invoke({'topic': 'ai'}))

# Conditional Logic (Branching)

**RunnableBranch** allows your chain to make decisions, similar to an "if-then-else" statement.

It evaluates a condition (like "is the text longer than 500 words?") and chooses which path to execute. This enables dynamic workflows that adapt to the input.

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Explain {topic} in siple words.',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Summarize this text in unter 500 words.\n Text: {text}',
    input_variables=['text']
)

chain1 = prompt1 | model | parser 
chain2 = RunnableBranch(
    (lambda x : len(x.split()) > 500, chain1 | prompt2 | model | parser),
    RunnablePassthrough()
)

chain3 = chain1 | chain2
result = chain3.invoke({'topic': 'AI'})

print(result)